# What does thid notebook do?  
using agglomerative clustering to find posible decision points in all notebooks  

# File mapping 
sklearn | clustering | ---: `./sklearn_dp.txt`

# Agglomerative Clustering

In [1]:
%matplotlib notebook

In [5]:
import os
from tqdm import tqdm
import numpy as np
from sklearn import cluster
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

import pdb
import json
import sklearn
import types

from importlib import import_module

from extract_func import process_file

In [3]:
class Tree(object):
    """tree sturcture to record currently found decision points"""
    def __init__(self, lib, funcs = []):
        """init a tree with a list of functions or []"""
        super(Tree, self).__init__()
        self.lib = lib
        self.root = {}
        for f in funcs:
            self.add_func(f)
    
    def add_func(self, func):
        """add function to tree"""
        tokens = func.split('.')
        cur_root = self.root
        for i, t in enumerate(tokens):
            if t not in cur_root:
                cur_root[t] = {}
            cur_root = cur_root[t]
            
    def add_funcs(self, funcs):
        """add functions to tree"""
        for f in funcs:
            self.add_func(f)

    def check_node(self, node):
        """delete leaves if very deep but not wide"""
        if len(node)<3:
            return {}
        else:
            node = {k: self.check_node(node[k]) for k in node}
            return node
    
    def cut_tree(self):
        self.cut_root = {self.lib: self.check_node(self.root[self.lib])}
        
    def tolist(self, prefix, tree):
        results = []
        for k in tree:
            if tree[k] =={}:
                results.append('{}.{}'.format(prefix,k) if prefix!='' else k)
            else:
                results+=self.tolist('{}.{}'.format(prefix,k) if prefix!='' else k, tree[k])
        return results
        
        

In [6]:
with open('./file_funcs.json','r') as f:
    data = json.load(f)

But what is the point to do clustering before choosing representative functions?
Why can't we just sort all the functions and choose short ones? 

In [7]:
# nb_path = '/projects/bdata/jupyter/target'
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

filter notebooks (which import sklearn)

In [8]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


# delete loops:
eg  
**sklearn.tree.DecisionTreeClassifier.fit.fit.fit**  
is equal to   
**sklearn.tree.DecisionTreeClassifier.fit**  
  
the only difference is that the first one is called multiple times  

In [9]:
def delete_loop(func):
    """
    delete continue loops in a function
    simplify **sklearn.tree.DecisionTreeClassifier.fit.fit.fit** 
          to **sklearn.tree.DecisionTreeClassifier.fit**
    """
    tokens = func.split('.')
    new_tokens = []
    for t in tokens:
        if new_tokens == [] or t!=new_tokens[-1]:
            new_tokens.append(t)
    return '.'.join(new_tokens)

In [10]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []
sklearn_counter = {}
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
#     try:
#         funcs, linenos = process_file(os.path.join(nb_path, nb))
#     except Exception as e:
#         err_files.append(nb)
    if nb in data:
        funcs = data[nb]["funcs"]
        linenos = data[nb]["linenos"]
#     funcs = [func for func in funcs if func in no_loop_sklearn_funcs]
    funcs = [func for func in funcs if func.startswith('sklearn.')]
    
    funcs = set(funcs)
    for func in funcs:
#         if not func.startswith('sklearn'):
#             continue
        if func not in func2vector:
            func2vector[func] = np.zeros(vector_size)
        if func not in sklearn_counter:
            sklearn_counter[func]=0
        func2vector[func][i] = 1
        sklearn_counter[func]+=1

Log: 0 notebooks processed
Log: 10000 notebooks processed
Log: 20000 notebooks processed
Log: 30000 notebooks processed
Log: 40000 notebooks processed
Log: 50000 notebooks processed
Log: 60000 notebooks processed
Log: 70000 notebooks processed
Log: 80000 notebooks processed
Log: 90000 notebooks processed


In [11]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [13]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [15]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.996, affinity="precomputed", linkage="average").fit(a)

In [16]:
model.n_clusters_

3464

In [17]:
new_clusters = {}
# new_clusters = [[]]*model.n_clusters_
for i, l in enumerate(model.labels_):
    if l not in new_clusters:
        new_clusters[l] = []
    new_clusters[l].append(idx2func[i])
#     print(i)
clusters = list(new_clusters.values())

In [18]:
clusters

[['sklearn.feature_extraction.text.TfidfVectorizer',
  'sklearn.feature_extraction.text.TfidfVectorizer.transform',
  'sklearn.feature_extraction.text.CountVectorizer.transform',
  'sklearn.feature_extraction.text.CountVectorizer.fit',
  'sklearn.feature_extraction.text.CountVectorizer',
  'sklearn.naive_bayes.MultinomialNB',
  'sklearn.feature_extraction.text.CountVectorizer.transform.toarray',
  'sklearn.naive_bayes.MultinomialNB.predict',
  'sklearn.feature_extraction.text.CountVectorizer.get_feature_names',
  'sklearn.feature_extraction.text.CountVectorizer.fit_transform',
  'sklearn.pipeline.Pipeline',
  'sklearn.model_selection.KFold',
  'sklearn.model_selection.KFold.split',
  'sklearn.naive_bayes.MultinomialNB.fit.predict',
  'sklearn.feature_extraction.text.CountVectorizer.fit_transform.tocsc.tocsc',
  'sklearn.naive_bayes.MultinomialNB.fit.predict_proba',
  'sklearn.feature_extraction.text.TfidfVectorizer.fit_transform',
  'sklearn.naive_bayes.MultinomialNB.fit.predict_log_pr

In [19]:
def is_parent_child(parent_func, child_func):
    if len(parent_func)>=len(child_func):
        return False
    if child_func.startswith(parent_func):
        if len(child_func.split('.'))>len(parent_func.split('.')):
            return True
        else:
            return False
    else:
        return False

In [20]:
not_root = []
sets = []
has_root = []

def find_root_in_cluster(cluster):
    """
    find a representative function in a cluster like:
     ['sklearn.preprocessing.MinMaxScaler',
      'sklearn.preprocessing.MinMaxScaler.fit_transform',
      'sklearn.preprocessing.MinMaxScaler.transform']
    return a list []
    """
#     if len(cluster)>3:
#         return []
    cluster = sorted(cluster)
    last_root = '*'
    cluster_roots = []
    for f in cluster:
        if not f.startswith(last_root):
            cluster_roots.append(f)
            last_root = f
        elif f not in has_root:
            has_root.append(f)
    cluster_roots = [r for r in cluster_roots if not any([is_parent_child(root, r) for root in roots])]
    return cluster_roots
            

    

In [21]:
roots = []
tree = Tree('sklearn')
for c in tqdm(clusters):
    funcs = find_root_in_cluster(c)
    roots+=funcs
    tree.add_funcs(funcs)
# print(json.dumps(tree.root, ensure_ascii=False, indent=2))
    

100%|██████████| 3464/3464 [00:00<00:00, 6255.05it/s]


In [22]:
sorted(roots)

['sklearn.GradientBoostingClassifier',
 'sklearn.GridSearchCV',
 'sklearn.LinearRegression',
 'sklearn.MLPClassifier',
 'sklearn.__version__.replace',
 'sklearn.__version__.split',
 'sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.CalibratedClassifierCV.predict',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.KMedoids',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.Meanshift',
 'sklearn.cluster.Meanshift.fit',
 'sklearn.cluster.Meanshift.predict',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.MiniBatchKMeans.predict',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.Ward',
 'sklearn

In [23]:
len(roots)

629

In [24]:
# 删去representatives 中还有sub 关系的函数（只相差一级的话）
dp = []
for f in sorted(roots):
    if dp==[] or len(dp[-1].split('.'))<3:
        dp.append(f)
    elif not is_parent_child(dp[-1], f):
        dp.append(f)
    else:
        continue

In [25]:
len(dp)

536

In [26]:
dp

['sklearn.GradientBoostingClassifier',
 'sklearn.GridSearchCV',
 'sklearn.LinearRegression',
 'sklearn.MLPClassifier',
 'sklearn.__version__.replace',
 'sklearn.__version__.split',
 'sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.KMedoids',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.Meanshift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.Ward',
 'sklearn.cluster.WardAgglomeration',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.append',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.biclu

In [27]:
from sklearn import *
# import sklearn.model_selection

In [28]:
# temp = dp
temp = []
for f in dp:
    tokens = f.split('.')
    try:
        getattr(sklearn, tokens[1])
#         mod = import_module('sklearn.{}'.format(tokens[1]))
#         getattr(mod, tokens[2])
#         print(f)
        temp.append(f)
    except Exception as e:
#         pass
        print(e)
# temp = new_temp
# temp
temp

module 'sklearn' has no attribute 'GradientBoostingClassifier'
module 'sklearn' has no attribute 'GridSearchCV'
module 'sklearn' has no attribute 'LinearRegression'
module 'sklearn' has no attribute 'MLPClassifier'
module 'sklearn' has no attribute 'cross_val_score'
module 'sklearn' has no attribute 'cross_val_score'
module 'sklearn' has no attribute 'cross_val_score'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_validation'
module 'sklearn' has no attribute 'cross_valid

['sklearn.__version__.replace',
 'sklearn.__version__.split',
 'sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.KMedoids',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.Meanshift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.Ward',
 'sklearn.cluster.WardAgglomeration',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.append',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.birch.Birch',
 'sklearn.cluster.data.argsort',
 'sklearn.cluster.estimate

In [30]:
# temp = dp
new_temp = []
for f in temp:
    tokens = f.split('.')
    try:
        getattr(sklearn, tokens[1])
        mod = import_module('sklearn.{}'.format(tokens[1]))
        getattr(mod, tokens[2])
#         print(f)
        new_temp.append(f)
    except Exception as e:
#         pass
        print(e)
# temp = new_temp
# temp
new_temp

No module named 'sklearn.__version__'
No module named 'sklearn.__version__'
No module named 'sklearn.clone'
module 'sklearn.cluster' has no attribute 'KMedoids'
module 'sklearn.cluster' has no attribute 'Meanshift'
module 'sklearn.cluster' has no attribute 'Ward'
module 'sklearn.cluster' has no attribute 'WardAgglomeration'
module 'sklearn.cluster' has no attribute 'append'
module 'sklearn.cluster' has no attribute 'data'
module 'sklearn.cluster' has no attribute 'value_counts'
module 'sklearn.datasets' has no attribute '___'
module 'sklearn.datasets' has no attribute 'boston'
module 'sklearn.datasets' has no attribute 'generate_linearly_separable_dataset'
module 'sklearn.datasets' has no attribute 'load_classification'
module 'sklearn.datasets' has no attribute 'load_dataset'
module 'sklearn.datasets' has no attribute 'load_mlcomp'
module 'sklearn.decomposition' has no attribute 'ProbabilisticPCA'
module 'sklearn.decomposition' has no attribute 'ProjectedGradientNMF'
module 'sklearn.d

['sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.birch.Birch',
 'sklearn.cluster.estimate_bandwidth',
 'sklearn.cluster.hierarchical.AgglomerativeClustering',
 'sklearn.cluster.k_means',
 'sklearn.cluster.spectral_clustering',
 'sklearn.covariance.EllipticEnvelope',
 'sklearn.covariance.EmpiricalCovariance',
 'sklearn.covariance.GraphLassoCV',
 'skl

In [32]:
import types

In [33]:
def search_func(mod, func):
    tokens = func.split('.')
    prefix = mod.__name__
#     pdb.set_trace()
    for t in tokens[1:]:
#         if t=='BaseEstimator':
#             pdb.set_trace()
#         print(t)
#         print(mod)
        sub = getattr(mod, t)
#         if isinstance(sub, module):
        prefix +='.{}'.format(t)
        if type(sub)==types.ModuleType or type(sub)==types.ClassType:
            mod = sub
        elif type(sub)==types.FunctionType:
#             prefix = sub
            break
        else:
            assert ValueError, type(sub)
    return prefix

In [35]:
final_dp = []
for f in new_temp:
    try:
        print('-'*20)
        corrected_func = search_func(sklearn, f)
        if f!=corrected_func:
            
            print(f)
#             print(search_func(sklearn, f))
            print(corrected_func)
        final_dp.append(corrected_func)
    except Exception as e:
        print(f)
        print(e)
        

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
-------------

In [38]:
final_dp

['sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.birch.Birch',
 'sklearn.cluster.estimate_bandwidth',
 'sklearn.cluster.hierarchical.AgglomerativeClustering',
 'sklearn.cluster.k_means',
 'sklearn.cluster.spectral_clustering',
 'sklearn.covariance.EllipticEnvelope',
 'sklearn.covariance.EmpiricalCovariance',
 'sklearn.covariance.GraphLassoCV',
 'skl

# Expansion

In [ ]:
# extract structure from statsmodels_roots
statsmodels_roots = ['.'.join(f.split('.')[:-1]) for f in sorted(tree.tolist('',tree.cut_root))]

In [ ]:
roots_counter = {}
for r in statsmodels_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
    
    
frequent_roots = []
for r in roots_counter:
    try:
        mod = import_module(r)
        if  roots_counter[r]/len(dir(mod))>0.1:
            frequent_roots.append(r)
    except:
        pass
    
for r in frequent_roots:
    try:
        mod = import_module(r)
        try:
            dps = ['{}.{}'.format(r, obj) for obj in mod.__all__] 
        except:
            dps =  ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
        statsmodels_dp+=dps
    except Exception as e:
        print(e)
statsmodels_dp = sorted(list(set(statsmodels_dp)))

In [ ]:
len(statsmodels_dp)

In [ ]:
for f in statsmodels_dp:
    print(f)

In [ ]:
v1 = func2vector["sklearn.cluster.Birch.fit"]
v2 = func2vector["sklearn.cluster.Birch"]
v3 = func2vector["sklearn.cluster.AgglomerativeClustering.labels_.astype"]
np.linalg.norm(v1- v3)